## Imports

In [1]:
from constants import FLUXES_FOLDER, SLM_FOLDER

from data_utils import load_numpy_data, \
                       flatten_data, \
                       shuffle_arrays, \
                       fuse_amplitude_and_phase

from plot_utils import plot_map, \
                       plot_model_history

from modeling_utils import create_linear_architecture_for_amplitude_reconstruction, \
                           compile_linear_model_for_amplitude_reconstruction, \
                           train_linear_model_for_amplitude_reconstruction

2023-11-23 11:46:45.660312: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-23 11:46:46.364473: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Load data

In the first place, define the data paths

In [2]:
fluxes_filename = "all_fluxes.npy"
amplitudes_filename = "complexsine_pupamp.npy"
phases_filename = "complexsine_pupphase.npy"

fluxes_path = f"{FLUXES_FOLDER}/{fluxes_filename}"
amplitudes_path = f"{SLM_FOLDER}/{amplitudes_filename}"
phases_path = f"{SLM_FOLDER}/{phases_filename}"

Read the numpy raw files

In [3]:
n_points = 10000
fluxes_array = load_numpy_data(fluxes_path,
                               n_points)

amplitudes_array = load_numpy_data(amplitudes_path,
                                   n_points)

phases_array = load_numpy_data(phases_path,
                               n_points)

Shuffle the data

In [4]:
fluxes_array, amplitudes_array, phases_array = shuffle_arrays([fluxes_array,
                                                               amplitudes_array,
                                                               phases_array])

Merge phase and amplitude

In [5]:
amp_phase_array = fuse_amplitude_and_phase(amplitudes_array,
                                           phases_array)

Split data in train and validation set

In [ ]:
val_ratio = 0.1
train_fluxes_array, val_fluxes_array = split_data(fluxes_array,
                                                  val_ratio)
